In [1]:
%load_ext rich

# Pre-built modules

In [2]:
import promptimus as pm

In [3]:
ollama = pm.llms.OpenAILike(
    model_name="gemma3:4b", base_url="http://lilan:11434/v1", api_key="DUMMY"
)

## Memory Module

In [4]:
assistant = pm.modules.MemoryModule(
    memory_size=3, system_prompt="You are an assistant."
).with_llm(ollama)

In [5]:
await assistant.forward("Hi my name is Ailadin!")


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content='Hi Ailadin! It’s lovely to meet you. What’s on your mind today? 😊 \n\nDo you want to chat about something specific, or were you just saying hello?',
    tool_calls=None,
    tool_call_id=None
)

In [6]:
# remembering name from context
await assistant.forward("What is my name?")


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content="Your name is Ailadin! 😊 I just confirmed it with you. \n\nIt's nice to chat with you. \n\nIs there anything you’d like to talk about?",
    tool_calls=None,
    tool_call_id=None
)

In [7]:
await assistant.forward("What is your name?")


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content='That’s a great question! I don’t really *have* a name in the way a person does. I’m a large language model, and I was developed by Google. You can just think of me as an assistant. 😊 \n\nWhat can I help you with today?',
    tool_calls=None,
    tool_call_id=None
)

In [8]:
await assistant.forward("What is my the weather today?")


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content='Okay, to tell you the weather, I need to know your location! 😊 \n\nCould you tell me:\n\n*   **What city and state (or country) are you in?** \n\nOnce you give me that information, I can look up the weather for you.',
    tool_calls=None,
    tool_call_id=None
)

In [9]:
# memory maxlen exhausted
assistant.memory

Memory[deque([Message(role=<MessageRole.ASSISTANT: 'assistant'>, content='That’s a great question! I don’t really *have* a name in the way a person does. I’m a large language model, and I was developed by Google. You can just think of me as an assistant. 😊 \n\nWhat can I help you with today?', tool_calls=None, tool_call_id=None), Message(role=<MessageRole.USER: 'user'>, content='What is my the weather today?', tool_calls=None, tool_call_id=None), Message(role=<MessageRole.ASSISTANT: 'assistant'>, content='Okay, to tell you the weather, I need to know your location! 😊 \n\nCould you tell me:\n\n*   **What city and state (or country) are you in?** \n\nOnce you give me that information, I can look up the weather for you.', tool_calls=None, tool_call_id=None)], maxlen=3)]

In [10]:
# now it doesn't have memory in context
await assistant.forward("Can you recall my name?")


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content="I apologize, but I have no memory of our previous conversations. As an AI, I don't retain information from one interaction to the next. \n\nPlease don't tell me your name again – it's not something I can remember! 😊 \n\nLet's start fresh. How can I help you today?",
    tool_calls=None,
    tool_call_id=None
)

In [11]:
# memory can be cleaned manually
assistant.memory.reset()
assistant.memory

Memory[deque([], maxlen=3)]

In [12]:
# or can be cleared on context enter and exit

with assistant.memory:
    await assistant.forward("Hi my name is Ailadin")
    await assistant.forward("What is my name")
    print("Inside context", assistant.memory)
print("----")
print("Outside context", assistant.memory)

Inside context Memory[deque([Message(role=<MessageRole.ASSISTANT: 'assistant'>, content='Hi Ailadin! It’s really nice to meet you. How can I help you today? 😊 \n\nDo you have something specific you’d like to talk about, or were you just saying hello?', tool_calls=None, tool_call_id=None), Message(role=<MessageRole.USER: 'user'>, content='What is my name', tool_calls=None, tool_call_id=None), Message(role=<MessageRole.ASSISTANT: 'assistant'>, content='Your name is Ailadin! 😊 \n\nIt’s a lovely name. \n\nIs there anything you’d like to tell me about yourself, or were you just checking to see if I remembered?', tool_calls=None, tool_call_id=None)], maxlen=3)]
----
Outside context Memory[deque([], maxlen=3)]


In [13]:
print(assistant.describe())


[prompt]
prompt = """
You are an assistant.
"""

role = """
system
"""




## Structural Output

In [4]:
from enum import StrEnum, auto

from pydantic import BaseModel, Field

In [5]:
class Operation(StrEnum):
    SUM = auto()
    SUB = auto()
    DIV = auto()
    MUL = auto()


class CalculatorSchema(BaseModel):
    reasoning: str
    a: float = Field(description="The left operand.")
    b: float = Field(description="The right operand.")
    op: Operation = Field(description="The operation to execute.")


ollama = pm.llms.OpenAILike(
    model_name="gemma3:12b", base_url="http://lilan:11434/v1", api_key="DUMMY"
)

module = pm.modules.StructuralOutput(
    CalculatorSchema, retry_message_role=pm.MessageRole.USER
).with_llm(ollama)

In [17]:
await module.forward("I have 10 cows, I need twice the amount")


CalculatorSchema(
    reasoning='To find twice the amount of cows, I multiply the initial number of cows by 2.',
    a=10.0,
    b=2.0,
    op=<Operation.MUL: 'mul'>
)

In [18]:
await module.forward("I have 10 cows, i need half of them")

CalculatorSchema(reasoning='Half of 10 is 5.', a=10.0, b=0.5, op=<Operation.DIV: 'div'>)

In [19]:
await module.forward("I have 11.11 cows. and 11 left the herd")

CalculatorSchema(reasoning='11.11 - 11 = 0.11', a=11.11, b=11.0, op=<Operation.SUB: 'sub'>)

In [20]:
await module.forward("I have 5 cows. and my friend have 7")


CalculatorSchema(
    reasoning='To find the total number of cows, we need to sum the number of cows I have and the number my friend has.',
    a=5.0,
    b=7.0,
    op=<Operation.SUM: 'sum'>
)

In [21]:
print(module.describe())

model_json_schema = """
{
    "$defs": {
        "Operation": {
            "enum": [
                "sum",
                "sub",
                "div",
                "mul"
            ],
            "title": "Operation",
            "type": "string"
        }
    },
    "properties": {
        "reasoning": {
            "title": "Reasoning",
            "type": "string"
        },
        "a": {
            "description": "The left operand.",
            "title": "A",
            "type": "number"
        },
        "b": {
            "description": "The right operand.",
            "title": "B",
            "type": "number"
        },
        "op": {
            "$ref": "#/$defs/Operation",
            "description": "The operation to execute."
        }
    },
    "required": [
        "reasoning",
        "a",
        "b",
        "op"
    ],
    "title": "CalculatorSchema",
    "type": "object"
}
"""

retry_template = """
Your response does not conform to the required schema. P

### OpenAI provider

OpenAI provider supports structural output feature without needing to provide json schema in prompt.

In [6]:
openai = pm.llms.OpenAILike(
    model_name="gpt-4.1-nano",
)

In [7]:
module = pm.modules.StructuralOutput(CalculatorSchema, native=True).with_llm(openai)

In [8]:
print(module.describe())

model_json_schema = """
{
    "$defs": {
        "Operation": {
            "enum": [
                "sum",
                "sub",
                "div",
                "mul"
            ],
            "title": "Operation",
            "type": "string"
        }
    },
    "properties": {
        "reasoning": {
            "title": "Reasoning",
            "type": "string"
        },
        "a": {
            "description": "The left operand.",
            "title": "A",
            "type": "number"
        },
        "b": {
            "description": "The right operand.",
            "title": "B",
            "type": "number"
        },
        "op": {
            "$ref": "#/$defs/Operation",
            "description": "The operation to execute."
        }
    },
    "required": [
        "reasoning",
        "a",
        "b",
        "op"
    ],
    "title": "CalculatorSchema",
    "type": "object"
}
"""

retry_template = """
Your response does not conform to the required schema. P

In [9]:
await module.forward("I have 10 cows, I need twice the amount")


CalculatorSchema(
    reasoning='You currently have 10 cows and want twice that amount. Doubling 10 results in 20 cows.',
    a=10.0,
    b=10.0,
    op=<Operation.MUL: 'mul'>
)

In [10]:
await module.forward("I have 10 cows, i need half of them")


CalculatorSchema(
    reasoning='Half of 10 cows is obtained by dividing the total number of cows by 2. So, 10 divided by 2 equals 5.',
    a=10.0,
    b=2.0,
    op=<Operation.DIV: 'div'>
)

In [11]:
await module.forward("I have 11.11 cows. and 11 left the herd")


CalculatorSchema(
    reasoning='The initial number of cows is 11.11. When 11 cows leave the herd, the remaining cows are calculated by subtracting 11 from 11.11.',
    a=11.11,
    b=11.0,
    op=<Operation.SUB: 'sub'>
)

In [12]:
await module.forward("I have 5 cows. and my friend have 7")


CalculatorSchema(
    reasoning='The user provided the number of cows they have and their friend has. The task appears to be to sum these quantities.',
    a=5.0,
    b=7.0,
    op=<Operation.SUM: 'sum'>
)